In [ ]:
import numpy as np
import os
import PIL
import tensorflow as tf
from PIL import Image
 
im = Image.open("0021C_S.png")
pix = np.array(im)
print(pix.shape)


#set the dimensions
#180 both
im_ht = 180
im_wd = 180
bat_sz = 16
#training set
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/zachderse//Desktop/cross_sectional_data/datasets/', 
    batch_size = bat_sz,
    #color_mode='grayscale',
    seed = 123, 
    validation_split = .2, 
    image_size = (im_ht,im_wd),
    subset = "training")

#test set
ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/zachderse//Desktop/cross_sectional_data/datasets/', 
    batch_size = bat_sz,
    #color_mode='grayscale',
    seed = 123, 
    validation_split = .2, 
    image_size = (im_ht,im_wd),
    subset = "validation")

In [ ]:
#check correct class names
import matplotlib.pyplot as plt

class_names = ds_train.class_names
print(class_names)



for image_batch, labels_batch in ds_train:
    print(image_batch.shape)
    print(labels_batch.shape)
    break


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.cache().prefetch(buffer_size=AUTOTUNE)


normalization_layer = tf.keras.layers.Rescaling(1./255)


normalized_ds = ds_train.map(lambda x, y: (normalization_layer(x), y))

image_batch, labels_batch = next(iter(normalized_ds))

first_image = image_batch[2]
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = 4

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(im_ht, im_wd, 3)),
  tf.keras.layers.Conv2D(16, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  #tf.keras.layers.Dense(num_classes)
  tf.keras.layers.Dense(4, activation = 'softmax')
])

#base_learning_rate = 0.0001

model.compile(
  optimizer='adam',
  #optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=base_learning_rate),
  #optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=base_learning_rate/10),
  #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  loss = 'sparse_categorical_crossentropy',
  metrics=['accuracy'])

model.summary()
epochs=10
#overfit at around 15 epochs
history = model.fit(
  ds_train,
  validation_data=ds_valid,
  epochs=epochs
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#different ways to find a better fit

data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(im_ht,
                                  im_wd,
                                  3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

model = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, name="outputs")
])


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 11
history = model.fit(
  ds_train,
  validation_data=ds_valid,
  epochs=epochs
)

#makes no difference to the model

In [ ]:
#Testing another model


model_builder = tf.keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = tf.keras.applications.xception.preprocess_input
decode_predictions = tf.keras.applications.xception.decode_predictions

last_conv_layer_name = "block14_sepconv2_act"


model2 = model_builder(weights="imagenet")


#maybe try SGD?
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model2.fit(
  ds_train,
  validation_data=ds_valid,
  epochs=5
)
model2.summary()
#still quite low in prediction, more tweaks needed.